# Microsoft OneNote | Microsoft OneNoteこのノートブックは、`OneNote`からドキュメントを読み込む方法を説明しています。> This notebook covers how to load documents from `OneNote`.## Prerequisites | 前提条件1. [Microsoft identity platform](https://learn.microsoft.com/en-us/azure/active-directory/develop/quickstart-register-app)の指示に従ってアプリケーションを登録してください。   > Register an application with the [Microsoft identity platform](https://learn.microsoft.com/en-us/azure/active-directory/develop/quickstart-register-app) instructions.2. 登録が完了すると、Azureポータルはアプリ登録の「概要」ペインを表示します。アプリケーション（クライアント）IDが表示されます。`client ID`とも呼ばれるこの値は、Microsoft identity platform内であなたのアプリケーションを一意に識別するためのものです。   > When registration finishes, the Azure portal displays the app registration's Overview pane. You see the Application (client) ID. Also called the `client ID`, this value uniquely identifies your application in the Microsoft identity platform.3. **項目1**で従うべき手順の中で、リダイレクトURIを`http://localhost:8000/callback`に設定することができます   > During the steps you will be following at **item 1**, you can set the redirect URI as `http://localhost:8000/callback`4. **項目1**での手順を実行する際に、アプリケーションシークレットセクションで新しいパスワード（`client_secret`）を生成してください。   > During the steps you will be following at **item 1**, generate a new password (`client_secret`) under Application Secrets section.5. この[ドキュメント](https://learn.microsoft.com/en-us/azure/active-directory/develop/quickstart-configure-app-expose-web-apis#add-a-scope)の指示に従って、あなたのアプリケーションに以下の`SCOPES`（`Notes.Read`）を追加してください。   > Follow the instructions at this [document](https://learn.microsoft.com/en-us/azure/active-directory/develop/quickstart-configure-app-expose-web-apis#add-a-scope) to add the following `SCOPES` (`Notes.Read`) to your application.6. `pip install msal` と `pip install beautifulsoup4` コマンドを使用して、msalパッケージとbs4パッケージをインストールする必要があります。   > You need to install the msal and bs4 packages using the commands `pip install msal` and `pip install beautifulsoup4`.7. 手順の最後には、以下の値を持っていなければなりません：   > At the end of the steps you must have the following values:* `CLIENT_ID`  > `CLIENT_ID`* `CLIENT_SECRET`  > `CLIENT_SECRET`## 🧑 Instructions for ingesting your documents from OneNote | 🧑 OneNoteからドキュメントを取り込むための指示### 🔑 Authentication | 🔑 認証デフォルトでは、`OneNoteLoader`は`CLIENT_ID`と`CLIENT_SECRET`の値がそれぞれ`MS_GRAPH_CLIENT_ID`と`MS_GRAPH_CLIENT_SECRET`という名前の環境変数として保存されていることを期待しています。これらの環境変数は、アプリケーションのルートにある`.env`ファイルを通じて、またはスクリプト内で以下のコマンドを使用して渡すことができます。> By default, the `OneNoteLoader` expects that the values of `CLIENT_ID` and `CLIENT_SECRET` must be stored as environment variables named `MS_GRAPH_CLIENT_ID` and `MS_GRAPH_CLIENT_SECRET` respectively. You could pass those environment variables through a `.env` file at the root of your application or using the following command in your script.```pythonos.environ['MS_GRAPH_CLIENT_ID'] = "YOUR CLIENT ID"os.environ['MS_GRAPH_CLIENT_SECRET'] = "YOUR CLIENT SECRET"```このローダーは、[*ユーザーに代わって*](https://learn.microsoft.com/en-us/graph/auth-v2-user?context=graph%2Fapi%2F1.0\&view=graph-rest-1.0)と呼ばれる認証方式を使用しています。これはユーザーの同意を得る2段階認証です。ローダーをインスタンス化すると、アプリが必要とする権限に同意するためにユーザーが訪れるべきURLを表示する呼び出しが行われます。その後、ユーザーはこのURLにアクセスし、アプリケーションに同意を与えなければなりません。次に、ユーザーは結果として表示されるページのURLをコピーしてコンソールに貼り付ける必要があります。その後、ログイン試行が成功した場合、メソッドはTrueを返します。> This loader uses an authentication called [*on behalf of a user*](https://learn.microsoft.com/en-us/graph/auth-v2-user?context=graph%2Fapi%2F1.0\&view=graph-rest-1.0). It is a 2 step authentication with user consent. When you instantiate the loader, it will call will print a url that the user must visit to give consent to the app on the required permissions. The user must then visit this url and give consent to the application. Then the user must copy the resulting page url and paste it back on the console. The method will then return True if the login attempt was successful.```pythonfrom langchain.document_loaders.onenote import OneNoteLoaderloader = OneNoteLoader(notebook_name="NOTEBOOK NAME", section_name="SECTION NAME", page_title="PAGE TITLE")```認証が完了すると、ローダーはトークン（`onenote_graph_token.txt`）を`~/.credentials/`フォルダに保存します。このトークンは、以前説明したコピー＆ペーストの手順を踏まずに後で認証に使用できます。このトークンを認証に使用するためには、ローダーのインスタンスを生成する際に`auth_with_token`パラメータをTrueに変更する必要があります。> Once the authentication has been done, the loader will store a token (`onenote_graph_token.txt`) at `~/.credentials/` folder. This token could be used later to authenticate without the copy/paste steps explained earlier. To use this token for authentication, you need to change the `auth_with_token` parameter to True in the instantiation of the loader.```pythonfrom langchain.document_loaders.onenote import OneNoteLoaderloader = OneNoteLoader(notebook_name="NOTEBOOK NAME", section_name="SECTION NAME", page_title="PAGE TITLE", auth_with_token=True)```また、トークンを直接ローダーに渡すこともできます。これは、他のアプリケーションによって生成されたトークンで認証したい場合に便利です。例えば、[Microsoft Graph Explorer](https://developer.microsoft.com/en-us/graph/graph-explorer)を使用してトークンを生成し、それをローダーに渡すことができます。> Alternatively, you can also pass the token directly to the loader. This is useful when you want to authenticate with a token that was generated by another application. For instance, you can use the [Microsoft Graph Explorer](https://developer.microsoft.com/en-us/graph/graph-explorer) to generate a token and then pass it to the loader.```pythonfrom langchain.document_loaders.onenote import OneNoteLoaderloader = OneNoteLoader(notebook_name="NOTEBOOK NAME", section_name="SECTION NAME", page_title="PAGE TITLE", access_token="TOKEN")```### 🗂️ Documents loader | 🗂️ ドキュメントローダー#### 📑 Loading pages from a OneNote Notebook | 📑 OneNoteノートブックからページを読み込む`OneNoteLoader`はOneDriveに保存されているOneNoteノートブックからページを読み込むことができます。特定のノートブック、特定のセクション、または特定のタイトルのページをそれぞれフィルタリングするために、`notebook_name`、`section_name`、`page_title`の任意の組み合わせを指定できます。例えば、OneDrive内の任意のノートブックにある`Recipes`というセクションに保存されているすべてのページを読み込みたい場合などです。> `OneNoteLoader` can load pages from OneNote notebooks stored in OneDrive. You can specify any combination of `notebook_name`, `section_name`, `page_title` to filter for pages under a specific notebook, under a specific section, or with a specific title respectively. For instance, you want to load all pages that are stored under a section called `Recipes` within any of your notebooks OneDrive.```pythonfrom langchain.document_loaders.onenote import OneNoteLoaderloader = OneNoteLoader(section_name="Recipes", auth_with_token=True)documents = loader.load()```#### 📑 Loading pages from a list of Page IDs | 📑 ページIDのリストからページを読み込む別の方法として、読み込みたい各ページの`object_ids`のリストを提供することも可能です。そのためには、[Microsoft Graph API](https://developer.microsoft.com/en-us/graph/graph-explorer)にクエリを実行して、あなたが興味を持っているすべてのドキュメントIDを見つけ出す必要があります。この[リンク](https://learn.microsoft.com/en-us/graph/onenote-get-content#page-collection)は、ドキュメントIDを取得するのに役立つエンドポイントのリストを提供しています。> Another possibility is to provide a list of `object_ids` for each page you want to load. For that, you will need to query the [Microsoft Graph API](https://developer.microsoft.com/en-us/graph/graph-explorer) to find all the documents ID that you are interested in. This [link](https://learn.microsoft.com/en-us/graph/onenote-get-content#page-collection) provides a list of endpoints that will be helpful to retrieve the documents ID.例えば、あなたのノートブックに保存されているすべてのページに関する情報を取得するには、`https://graph.microsoft.com/v1.0/me/onenote/pages` へリクエストを送る必要があります。興味のあるIDのリストを手に入れた後、以下のパラメータを使用してローダーをインスタンス化することができます。> For instance, to retrieve information about all pages that are stored in your notebooks, you need make a request to: `https://graph.microsoft.com/v1.0/me/onenote/pages`. Once you have the list of IDs that you are interested in, then you can instantiate the loader with the following parameters.```pythonfrom langchain.document_loaders.onenote import OneNoteLoaderloader = OneNoteLoader(object_ids=["ID_1", "ID_2"], auth_with_token=True)documents = loader.load()```